Import libraries

In [ ]:
import os
import base64
from pdf2image import convert_from_path
from openai import AzureOpenAI
from PIL import Image, ImageOps
import io
import numpy as np

Add api key, api version, api base url, and deployment name.

In [ ]:
api_key= ""
api_version = ""
api_base = "" 
deployment_name = ""  

Initialize the AzureOpenAI client

In [ ]:
client = AzureOpenAI(
    api_key=api_key,
    api_version=api_version,
    base_url=f"{api_base}/openai/deployments/{deployment_name}"
)

Helper functions for converting pdfs to jpg and images to base 64.